# Стадия I

Программа принимает на вход скорости и координаты космических объектов и рассчитывает их положение и скорость в зависимости от времени методом численного решения уравнений Ньютона. Обоснован выбор разностной схемы и шага интеграции. Требуемая точность - отклонение от аналитической орбиты до 10 cм за неделю на орбитах высотой от 300 до 1200 км для модели с единственной силой - точечной гравитацией. Если это не достигается на стандартных типах данных, докажите, обсудим. Расчёт на GPU

Все расчёты в системе единиц СИ. Чтобы так не получилось.

Есть код, генерирующий случайные начальные условия для 21000 объектов. Круговые орбиты, высота распределена по Гауссу, средняя 800 км, дисперсия 200 км, плоскости - равномерно. Размеры объектов от 1 см до 10 м, распределены с pdf = 1/object_size

В коде есть интерфейс для простого добавления новых сил, действующих на объекты. Силы могут зависеть от времени, скорости и координат объекта, параметров объекта. В нём реализована элементарная модель гравитации, где Земля - шар.

Есть визуализатор, для заданных id объектов и интервалу времени, рисующий их траекторию картинкой. В отчёте есть траектории 10 случайных объектов на протяжении суток.
